In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1879,2024-09-27,Hungria Nacional,14:00,Soproni KC,DEAC,1.46,2.47,153.5,1.88,1.78,-5.5,1.96,1.71,ruz51wXt,0.684932,0.404858,0.531915,0.561798,0.089790,246.506,208.740806,0.846798,1.2,1.643168,1.369306,128.16,3.122,2.475878,0.793042,-46.0,148.438,57.540939,0.387643,0.0,0.000000,NaN,230.55,2.146,0.608465,0.283535,-74.0,89,87,1.44,2.65,138.012,193.026,0.363449,0.038640,0.096336,-3.56,-0.712,-0.646067,0.553447,0.6,0.046553,-2.74,-0.548,-2.682482,0.438192,0.3,-0.138192
1880,2024-09-27,França Pro B,15:30,Vichy,St. Chamond,1.52,2.30,162.5,1.83,1.83,-4.5,1.96,1.72,r721r9l3,0.657895,0.434783,0.546448,0.546448,0.092677,129.694,27.922816,0.215298,1.8,1.643168,0.912871,114.80,1.534,0.444443,0.289728,19.0,193.102,88.127761,0.456379,0.6,1.341641,2.236068,181.76,2.816,1.236054,0.438940,-45.0,82,64,1.40,2.84,116.192,235.562,0.288766,0.000000,0.092231,0.20,0.040,13.000000,0.775588,0.8,0.024412,-2.53,-0.506,-2.569170,0.352948,0.1,-0.252948
1881,2024-09-27,França Pro B,15:50,Pau-Orthez,Antibes,1.34,2.87,147.5,1.83,1.83,-6.5,1.94,1.74,xd27QzTH,0.746269,0.348432,0.546448,0.546448,0.094701,156.462,41.535221,0.265465,1.8,1.643168,0.912871,214.42,2.138,0.617633,0.288883,-1.0,149.540,50.224354,0.335859,2.4,1.341641,0.559017,104.94,1.812,0.381602,0.210597,32.0,71,66,3.02,1.59,132.500,192.848,0.513954,0.000000,0.076859,1.58,0.316,1.075949,0.650704,0.7,0.049296,1.00,0.200,9.350000,0.449647,0.4,-0.049647
1882,2024-09-27,Hungria Nacional,14:00,Budapesti Honved Se,Atomeromu Paks,2.03,1.66,161.5,1.88,1.79,-2.5,2.03,1.67,ChDXJom8,0.492611,0.602410,0.531915,0.558659,0.095020,181.784,57.143636,0.314349,2.4,1.341641,0.559017,249.48,2.250,0.658901,0.292845,61.0,162.164,52.121355,0.321411,1.2,1.643168,1.369306,202.40,1.894,0.581790,0.307175,-17.0,84,80,2.97,2.53,222.300,240.650,0.141805,0.034681,0.137599,-0.18,-0.036,-28.611111,0.525999,0.7,0.174001,2.99,0.598,1.103679,0.368884,0.4,0.031116
1883,2024-09-27,Israel Copa Da Liga,06:40,Maccabi Ironi Ramat Gan,Bnei Herzliya,1.52,2.30,162.5,1.84,1.82,-4.5,1.91,1.75,Y9EFyf6s,0.657895,0.434783,0.543478,0.549451,0.092677,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.288766,0.007728,0.061824,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1884,2024-09-27,França Pro B,15:30,Hyeres-Toulon,Poitiers,2.76,1.37,159.5,1.83,1.83,4.5,2.01,1.68,8UkPrIrn,0.362319,0.729927,0.546448,0.546448,0.092246,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,180.164,74.264474,0.412205,0.6,1.341641,2.236068,122.24,2.544,1.124802,0.442139,-43.0,0,64,0.00,1.91,0.000,235.564,0.475970,0.000000,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,-1.75,-0.350,-1.057143,0.347668,0.3,-0.047668
1885,2024-09-27,México Lnbp,00:00,Soles,Panteras,1.74,1.99,167.5,1.84,1.82,-2.5,1.97,1.71,SY6hBCum,0.574713,0.502513,0.543478,0.549451,0.077225,187.236,39.270188,0.209736,1.8,1.643168,0.912871,171.36,2.198,0.660848,0.300659,15.0,160.044,25.732794,0.160786,1.2,1.643168,1.369306,147.92,1.930,0.461086,0.238904,7.0,102,86,1.68,1.72,0.000,202.102,0.094786,0.007728,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-0.990000,0.000000,0.0,0.000000
1886,2024-09-27,República

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
